# Hyperparameter Tuning with sklearn

In this tutorial, we will walk you through an example workflow for how you can use hyperparameter tuning with [sci-kit learn](https://scikit-learn.org/stable/) to pick the best model for your machine learning problem.

**Requirements:** Please add the `scikit-learn` package from the package picker on the top right. We will be using this package in the notebook.

## Load Iris Dataset

 We will be using sklearn's built-in [Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) to predict the flower type based on features, such as the length and width of the sepal and petal.

In [15]:
# Load iris flower dataset
import warnings
warnings.filterwarnings("ignore")

from sklearn import svm, datasets
iris = datasets.load_iris()

In [16]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Let's start by using `train_test_split` to manually tune parameters by trial and error.

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [18]:
from sklearn.model_selection import cross_val_score
# Build a Support Vector Classification (SVC) model
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9777777777777777

It's difficult to guess the right parameters to use for the model, so let's loop over different parameter values and use K Fold cross validation and compute the average score.

In [19]:
# Using a for-loop to iterate over different kernel types and regularization parameter (C) values
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

## Hyperparameter Search with Grid Search
sklearn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) provides a convenient way for us to perform the parameter search without having to write a for-loop.

In [20]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,5,10,20,30,50],
    'kernel': ['rbf','linear','poly']
}, cv=5, return_train_score=False)
gs.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 5, 10, 20, 30, 50],
                         'kernel': ['rbf', 'linear', 'poly']})

In [21]:
# Putting GridSearchCV result into a dataframe
df = pd.DataFrame(gs.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001393,0.000202,0.000857,0.000071,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001081,0.000079,0.000757,0.000054,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002464,0.001437,0.000823,0.000108,1,poly,"{'C': 1, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,8
3,0.001130,0.000091,0.000762,0.000028,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.000960,0.000035,0.000660,0.000011,5,linear,"{'C': 5, 'kernel': 'linear'}",1.000000,1.000000,0.933333,0.966667,1.0,0.980000,0.026667,1
5,0.003783,0.002253,0.000836,0.000114,5,poly,"{'C': 5, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,8
6,0.001234,0.000189,0.000851,0.000091,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
7,0.001125,0.000118,0.000743,0.000063,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,6
8,0.004516,0.002861,0.000869,0.000102,10,poly,"{'C': 10, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,8
9,0.001164,0.000067,0.000784,0.000027,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,7


In [22]:
results = df[['param_C','param_kernel','mean_test_score']]
results

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,1,poly,0.966667
3,5,rbf,0.980000
4,5,linear,0.980000
5,5,poly,0.966667
6,10,rbf,0.980000
7,10,linear,0.973333
8,10,poly,0.966667
9,20,rbf,0.966667


Let's plot the scores for each parameter combination in a heatmap to visualize the grid search results:

In [23]:
import altair as alt
alt.Chart(results).mark_rect().encode(
    x='param_C:O',
    y='param_kernel:O',
    color='mean_test_score:Q'
)

alt.Chart(...)

In [24]:
print(f"From GridSearch, this is the best parameter combination {gs.best_params_} and resulting score {gs.best_score_:.2f}.")

From GridSearch, this is the best parameter combination {'C': 1, 'kernel': 'rbf'} and resulting score 0.98.


## Hyperparameter Search with Randomized Search
GridSearchCV performs an exhaustive search over all parameter combinations. That can become very expensive when there is large numbers of parameters to search through.
[RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) performs a random search over the space of possible parameters.

In [25]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,5,10,20,30,50],
        'kernel': ['rbf','linear','poly']
    },
    cv=5,
    return_train_score=False,
    n_iter=2
)
import time
start = time.time()
rs.fit(iris.data, iris.target)
end = time.time()
print(f"RandomizedSearchCV took {end-start:.2f} seconds")
df = pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
df

RandomizedSearchCV took 0.08 seconds


,param_C,param_kernel,mean_test_score
0,50,rbf,0.960000
1,10,poly,0.966667


In [26]:
start = time.time()
gs.fit(iris.data, iris.target)
end = time.time()
print(f"GridSearchCV took {end-start:.2f} seconds")

GridSearchCV took 0.49 seconds


We can see that GridSearchCV takes longer than RandomizedSearchCV. The runtime difference is even larger when you have many parameters to try or if the training time is long (e.g., when you are working with large datasets). Read more about the difference between the two hyperparameter tuning methods [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html).

## Model Comparison with Grid Search

Now that we learned how we can find the optimized parameter for a given model, we can loop through different model and parameter ranges and perform grid search to determine the most optimal parameters for different types of classification models.

In [27]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,5,10,20,30,50],
            'kernel': ['rbf','linear','poly']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}


In [28]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}


Based on above, we can determine that SVM with C=1 and Radial Basis Function (RBF) kernel is the most accurate model for this iris flower classification task.

## Conclusion
In this example, we looked at how you can use scikit-learn to search over a range of parameter combinations and find the best parameter settings for your model using GridSearchCV and RandomizedSearchCV. We also looked at how you can apply this technique to search over different models to look for the best model-parameter combination for your machine learning task.